Got inspired [here](https://developers.google.com/earth-engine/tutorials/community/time-series-visualization-with-altair)

In [1]:
import ee
import geemap
from geemap import ee_initialize
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

ee_initialize()

In [2]:

from create_reduce_region_function import create_reduce_region_function
from fc_to_dict import fc_to_dict
from add_date_info import add_date_info
from addNDVI import addNDVI
from do_buffer import do_buffer

In [3]:
stations = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point(10.5798451965, 46.6862577024), {'name': 'B2 (1490m)'}),
  ee.Feature(ee.Geometry.Point(10.5918853513, 46.6917328627), {'name': 'B3 (1930m)'}),
  ee.Feature(ee.Geometry.Point(10.5850561988, 46.684249288), {'name': 'P2 (1549m)'})])

aoi = stations.map(do_buffer)
aoi_all = aoi.geometry()
aoi_sel = ee.Geometry.Point(10.5850561988, 46.684249288)

In [4]:
s2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2016-01-01', '2019-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .filterBounds(aoi_all)

In [5]:
ndvi = s2.map(addNDVI).select("NDVI")
ndvi_bands = ndvi.toBands().clip(aoi_all)

In [6]:
ndvi_vis = {
  'min': -1.0,'max': 1,
  'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', 
  '74A901','66A000', '529400', '3E8601', '207401', '056201', '004C00', 
  '023B01','012E01', '011D01', '011301']
}

Map = geemap.Map(center=[ 46.688,10.586], zoom = 15)
Map.add_basemap("SATELLITE")
Map.addLayer(ndvi_bands,name="NDVI_series")
Map.addLayer(ndvi_bands.select(4),ndvi_vis,name="NDVI_image")
Map.addLayer(stations,name="Monitoring Stations")
Map.set_plot_options(plot_type='bar',add_marker_cluster=False, marker=None)
Map.roi_reducer = ee.Reducer.mean()
Map

Map(center=[46.688, 10.586], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(…

## NDVI for P2

In [117]:
reduce_ndvi = create_reduce_region_function(
    geometry=aoi_sel, reducer=ee.Reducer.median(), scale=10)

ndvi_stat_fc = ee.FeatureCollection(ndvi.map(reduce_ndvi)).filter(
    ee.Filter.notNull(ndvi.first().bandNames()))

ndvi_dict = fc_to_dict(ndvi_stat_fc).getInfo()
ndvi_df = pd.DataFrame(ndvi_dict)

ndvi_df = add_date_info(ndvi_df).drop(['millis','system:index'],axis=1)
ndvi_df.head(5)

,NDVI,Timestamp,Year,Month,Day,DOY
0,0.020199,2016-01-03 10:33:20.407,2016,1,3,3
1,0.020199,2016-01-03 10:33:20.407,2016,1,3,3
2,0.532161,2016-05-22 10:20:29.455,2016,5,22,143
3,0.532161,2016-05-22 10:20:29.000,2016,5,22,143
4,0.707066,2016-06-28 10:10:26.461,2016,6,28,180


In [120]:
fig = px.line(ndvi_df, x="DOY", y="NDVI", color='Year')#
f = go.FigureWidget(fig)
f

FigureWidget({
    'data': [{'hovertemplate': 'Year=2016<br>DOY=%{x}<br>NDVI=%{y}<extra></extra>',
           …

## Precipitation for P2

In [115]:
prec = ee.ImageCollection('NASA/GPM_L3/IMERG_MONTHLY_V06') \
                  .filter(ee.Filter.date('2016-01-01', '2020-11-15'))

reduce_prec = create_reduce_region_function(
    geometry=aoi_sel, reducer=ee.Reducer.median(), scale=5000)

prec_stat_fc = ee.FeatureCollection(prec.map(reduce_prec)).filter(
    ee.Filter.notNull(prec.first().bandNames()))

prec_dict = fc_to_dict(prec_stat_fc).getInfo()
prec_df = pd.DataFrame(prec_dict)

prec_df = add_date_info(prec_df).drop(['millis','system:index'],axis=1)
prec_df['precipitation'] = prec_df['precipitation']*1000
prec_df.head(5)



,gaugeRelativeWeighting,precipitation,precipitationQualityIndex,probabilityLiquidPrecipitation,randomError,Timestamp,Year,Month,Day,DOY
0,99,131.115004,352.609406,10,0.003737,2016-01-01,2016,1,1,1
1,99,135.038778,539.945496,17,0.003079,2016-02-01,2016,2,1,32
2,99,63.604720,500.030945,27,0.001985,2016-03-01,2016,3,1,61
3,99,127.969444,543.810181,69,0.002962,2016-04-01,2016,4,1,92
4,99,196.306422,460.459106,88,0.004282,2016-05-01,2016,5,1,122


In [116]:
fig_prec = px.bar(prec_df, x="Timestamp", y="precipitation", color='Year')
f_prec = go.FigureWidget(fig_prec)
f_prec

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Timestamp=%{x}<br>preci…

## Land Surface Temperature (Day) for P2

In [ ]:
lst = ee.ImageCollection('MODIS/006/MOD11A2') \
                  .filter(ee.Filter.date('2016-01-01', '2020-11-15'))

reduce_lst = create_reduce_region_function(
    geometry=aoi_sel, reducer=ee.Reducer.median(), scale=5000)

lst_stat_fc = ee.FeatureCollection(lst.map(reduce_lst)).filter(
    ee.Filter.notNull(lst.first().bandNames()))

lst_dict = fc_to_dict(lst_stat_fc).getInfo()
lst_df = pd.DataFrame(lst_dict)

lst_df = add_date_info(lst_df).drop(['millis','system:index'],axis=1)
lst_df['LST_Day_1km'] = lst_df['LST_Day_1km']/1000
lst_df['LST_Night_1km'] = lst_df['LST_Night_1km']/1000
lst_df.head(5)

,Clear_sky_days,Clear_sky_nights,Day_view_angl,Day_view_time,Emis_31,Emis_32,LST_Day_1km,LST_Night_1km,Night_view_angl,Night_view_time,QC_Day,QC_Night,Timestamp,Year,Month,Day,DOY
0,1,249,41,109,247,249,13.839,13.310,74,220,65,0,2016-01-01,2016,1,1,1
1,128,68,109,118,247,249,13.508,13.154,90,224,65,65,2016-01-09,2016,1,9,9
2,226,121,88,114,247,249,13.720,13.244,71,220,65,0,2016-01-17,2016,1,17,17
3,177,19,99,117,247,248,14.094,13.671,60,218,65,0,2016-01-25,2016,1,25,25
4,102,73,70,110,248,249,13.729,13.355,49,217,65,0,2016-02-10,2016,2,10,41


In [102]:
fig_lst = px.line(lst_df, x="Timestamp", y="LST_Day_1km", color='Year')
f_lst = go.FigureWidget(fig_lst)
f_lst

FigureWidget({
    'data': [{'hovertemplate': 'Year=2016<br>Timestamp=%{x}<br>LST_Day_1km=%{y}<extra></extra>'…